In [3]:
import pandas as pd
import os
import ast

In [5]:
def load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    'category', categories=SUBSETS, ordered=True)
        except (ValueError, TypeError):
            # the categories and ordered arguments were removed in pandas 0.25
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks


In [12]:
tracks = load("../../../fma_metadata/tracks.csv")

In [13]:
tracks['track']

,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,
2,256000,0,NaN,2008-11-26 01:48:12,2008-11-26,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,256000,0,NaN,2008-11-26 01:48:14,2008-11-26,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
5,256000,0,NaN,2008-11-26 01:48:20,2008-11-26,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
10,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
20,256000,0,NaN,2008-11-26 01:48:56,2008-01-01,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155316,320000,0,NaN,2017-03-30 15:23:34,NaT,162,1,Rock,[25],"[25, 12]",NaN,122,NaN,Creative Commons Attribution-NonCommercial-NoD...,102,NaN,3,NaN,[],The Auger
155317,320000,0,NaN,2017-03-30 15:23:36,NaT,217,1,Rock,[25],"[25, 12]",NaN,194,NaN,Creative Commons Attribution-NonCommercial-NoD...,165,NaN,4,NaN,[],Let's Skin Ruby
155318,320000,0,NaN,2017-03-30 15:23:37,NaT,404,2,Rock,[25],"[25, 12]",NaN,214,NaN,Creative Commons Attribution-NonCommercial-NoD...,168,NaN,6,NaN,[],My House Smells Like Kim Deal/Pulp


In [22]:
classified_top_genre = pd.read_csv("../top_genre_prob_by_optimized_classification.csv",index_col=[0])

In [26]:
classified_remaining_genre = pd.read_csv("../reclassified_remaining_genres.csv",index_col=[0])

In [28]:
all_genre_probs = pd.concat([classified_top_genre,classified_remaining_genre])

In [30]:
track_recommendation_data = pd.DataFrame(tracks['track'][["interest","favorites","listens"]])

In [32]:
track_recommendation_data

,interest,favorites,listens
track_id,,,
2,4656,2,1293
3,1470,1,514
5,1933,6,1151
10,54881,178,50135
20,978,0,361
...,...,...,...
155316,122,1,102
155317,194,1,165
155318,214,2,168


In [34]:
recommendation_data = all_genre_probs.merge(track_recommendation_data,left_index=True,right_index=True)

In [36]:
# recommendation_data.to_csv("optimized_recommendation_data.csv")

In [38]:
recommendation_data

,Rock,Electronic,Pop,Hip-Hop,Folk,interest,favorites,listens
track_id,,,,,,,,
2,0.003639,0.010459,0.009752,0.974635,0.001515,4656,2,1293
3,0.045891,0.044842,0.106042,0.785288,0.017937,1470,1,514
5,0.005435,0.020162,0.036643,0.930348,0.007411,1933,6,1151
10,0.162003,0.133598,0.663075,0.021528,0.019796,54881,178,50135
134,0.030814,0.061230,0.042618,0.854423,0.010914,1126,3,943
...,...,...,...,...,...,...,...,...
155308,0.783553,0.067902,0.105080,0.019759,0.023706,90,1,82
155309,0.351662,0.071414,0.294607,0.037861,0.244455,90,0,79
155310,0.411248,0.029858,0.400344,0.057910,0.100640,94,0,84


In [40]:
recommendation_data.to_csv("final_recommendation_data.csv")